In [81]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [2]:
spark = SparkSession.builder.master("local[1]").appName('Spark Example').getOrCreate()

23/05/12 08:43:11 WARN Utils: Your hostname, ashish resolves to a loopback address: 127.0.1.1; using 172.18.222.152 instead (on interface eth0)
23/05/12 08:43:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/12 08:43:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [96]:
df = spark.read.json('../input_data/starter/transactions/*/transactions.json')

In [97]:
df.show()

+--------------------+-----------+-------------------+
|              basket|customer_id|   date_of_purchase|
+--------------------+-----------+-------------------+
|       [{1306, P37}]|         C4|2018-12-05 05:50:00|
|[{252, P06}, {926...|         C7|2018-12-05 13:58:00|
|          [{3, P57}]|         C8|2018-12-05 03:09:00|
|[{1316, P22}, {71...|        C21|2018-12-05 05:28:00|
|[{1800, P08}, {95...|        C22|2018-12-05 15:35:00|
|[{1075, P61}, {12...|        C23|2018-12-05 03:11:00|
|[{1089, P38}, {60...|        C30|2018-12-05 17:58:00|
|[{995, P26}, {238...|        C32|2018-12-05 17:50:00|
|       [{1928, P29}]|        C35|2018-12-05 23:39:00|
|[{901, P39}, {118...|        C36|2018-12-05 18:59:00|
|        [{243, P35}]|        C42|2018-12-05 09:55:00|
|[{362, P41}, {144...|        C43|2018-12-05 03:32:00|
|        [{201, P37}]|        C48|2018-12-05 05:55:00|
|        [{662, P45}]|        C50|2018-12-05 22:16:00|
|[{927, P61}, {145...|        C53|2018-12-05 10:08:00|
|       [{

In [44]:
df2 = df.select(df.customer_id, explode(df.basket).alias('nested'))
df2.show()

+-----------+-----------+
|customer_id|     nested|
+-----------+-----------+
|         C4|{1306, P37}|
|         C7| {252, P06}|
|         C7| {926, P06}|
|         C8|   {3, P57}|
|        C21|{1316, P22}|
|        C21| {714, P28}|
|        C21|{1147, P33}|
|        C22|{1800, P08}|
|        C22| {952, P12}|
|        C22|{1781, P11}|
|        C23|{1075, P61}|
|        C23|{1252, P61}|
|        C23| {266, P61}|
|        C30|{1089, P38}|
|        C30| {605, P35}|
|        C30|{1095, P35}|
|        C32| {995, P26}|
|        C32| {238, P29}|
|        C32| {568, P31}|
|        C35|{1928, P29}|
+-----------+-----------+
only showing top 20 rows



In [45]:
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- nested: struct (nullable = true)
 |    |-- price: long (nullable = true)
 |    |-- product_id: string (nullable = true)



In [55]:
trans = df2.withColumn('product_id', df2.nested.product_id).withColumn('price', df2.nested.price).drop('nested')
trans.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- price: long (nullable = true)



In [56]:
cust = spark.read.option("header", True).csv("../input_data/starter/customers.csv")
cust.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- loyalty_score: string (nullable = true)



In [57]:
prod = spark.read.option("header", True).csv("../input_data/starter/products.csv")
prod.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- product_category: string (nullable = true)



In [87]:
final = trans.join(cust, trans.customer_id==cust.customer_id, 'inner')\
.join(prod, trans.product_id==prod.product_id, 'inner')\
.select(trans.customer_id, cust.loyalty_score, trans.product_id, prod.product_category)\
.withColumn('purchase_count', count(col('customer_id')).over(Window.partitionBy(col('customer_id'))))

In [92]:
final.write.option("header",True) \
 .csv("../output_data/final.csv")